[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mongodb-developer/security-lab-notebooks/blob/main/template_notebook.ipynb)
### 1. Install atlas CLI on the notebook environment.

## Setup

In [ ]:
#Install atlas-cli

!wget https://fastdl.mongodb.org/mongocli/mongodb-atlas-cli_1.37.0_linux_x86_64.deb
!dpkg -i mongodb-atlas-cli_1.37.0_linux_x86_64.deb

# If you require sudo privilege, run the following instead:
# !sudo dpkg -i mongodb-atlas-cli_1.37.0_linux_x86_64.deb

### 2. Authenticate the CLI against your Atlas organization.

`atlas auth login` typically enters interactive mode. 
We are piping a new line character so that it selects a default organization for you. 

In [ ]:
# Authenticate with Atlas CLI
!echo -e "\n" | atlas auth login --noBrowser

### 3. Create a new project called \"MySecureProj\"

Replace `<ORGANIZATION_ID>` with the Organization ID from the previous step.

In [ ]:
# Retain the quotes ("") when pasting the Organization ID
org_id = "<ORGANIZATION_ID>"
!atlas projects create MySecureProj --orgId {org_id}

### 4. Create your challenge cluster

Replace `<PROJECT_ID>` with the Project ID from the previous step.

In [ ]:
# Retain the quotes ("") when pasting the Project ID
project_id = "<PROJECT_ID>"
# You may select a region closer to you instead of "US_EAST_1"
!atlas clusters create MyNewCluster --provider AWS --region US_EAST_1 --tier M0 --projectId {project_id}

## Network Challenge

### 5. Add 'My current IP' temporary into the atlas project

Refer to documentations : [atlas accessList](https://www.mongodb.com/docs/atlas/cli/current/command/atlas-accessLists/#std-label-atlas-accessLists)

In [ ]:
from datetime import datetime, timedelta
# Calculate the date and time 24 hours from now
delete_after = (datetime.utcnow() + timedelta(hours=24)).isoformat() + 'Z'
!atlas accessLists <CODE_BLOCK> --projectId {project_id} --deleteAfter {delete_after}

### 6. Check that 'My current IP' was added:

In [ ]:
!atlas accessLists list --output json --projectId {project_id}

## Authentication Challenge

### 7. Let's create a SCRAM user: 'myUser' with User/Password authentication and assign it the 'readWriteAnyDatabase' role.

Refer to documentations: [atlas dbusers](https://www.mongodb.com/docs/atlas/cli/current/command/atlas-dbusers-create/)

In [ ]:
# create a SCRAM user with username: "myUser", password: "mySecurePassword" and role: "readWriteAnyDatabase"
username = "myUser"
password = "mySecurePassword"
!atlas dbusers create <CODE_BLOCK> --projectId {project_id}

### 8. Lets test our SCRAM user successful creation by performing the authentication process

In [ ]:
!pip install pymongo dnspython

In [ ]:
# retrieve connection string
connection = !atlas clusters connectionStrings describe  MyNewCluster --projectId {project_id}

# add username and password to connection string
new_connection = connection[1].replace('mongodb+srv://', f'mongodb+srv://{username}:{password}@')
print(new_connection)

#make the connection get the list of databases
from pymongo import MongoClient
client = MongoClient(new_connection)
client.list_database_names()

### 10. Create a X509 user and certificate

Refer to documentations: [atlas dbusers](https://www.mongodb.com/docs/atlas/cli/current/command/atlas-dbusers-create/)

In [ ]:
# create a Atlas-managed X509 user with username: "myX509User" and role: "readAnyDatabase" 
!atlas dbusers create <CODE_BLOCK> --projectId {project_id}

Refer to documentations: [atlas dbusers certs](https://www.mongodb.com/docs/atlas/cli/current/command/atlas-dbusers-certs-create/)

In [ ]:
# Generate a certification for "myX509user", set monthsUntilExpiration to 1, and save it to /tmp/cert.pem
!atlas dbusers certs create <CODE_BLOCK> --projectId {project_id} > /tmp/cert.pem

### 11. Let's test our X509 User

In [ ]:
# Get connection string
connection = !atlas clusters connectionStrings describe MyNewCluster --projectId {project_id}

# Modify connection string to use X509 as authentication mechanism 
new_connection = connection[1].replace('.net', '.net?authSource=%24external&authMechanism=MONGODB-X509')
print(new_connection)

# Connect using the certificate
from pymongo import MongoClient
client = MongoClient(new_connection,
                     tlsCertificateKeyFile='/tmp/cert.pem')

# Access the database
client.list_database_names()

## RBAC Challenge

### 12. Create a user for "MyNewCluster" database only


In [ ]:
# Create a user:'myNewClusterAdmin', password:'myNewClusterAdminPass', role: 'readWriteAnyDatabase'
# and scoped to "MyNewCluster" database
newClusterAdminUser = 'myNewClusterAdmin'
newClusterAdminPass = 'myNewClusterAdminPass'
!atlas dbusers create <CODE_BLOCK> --scope <CLUSTER_NAME> --projectId {project_id}

### 13. Create user with read-only access to the 'salesDB' database

In [ ]:
#Create a role "salesRead" which access to read-only role to salesDB database
!atlas customDbRoles create <CODE_BLOCK>

#Create a user "salesReadUser" with password "salesReadPass" which has the "salesRead" role
salesReadUser = 'salesReadUser'
salesReadPass = 'salesReadPass'
!atlas dbusers create <CODE_BLOCK> --projectId {project_id}

### 14. Test that 'salesReadUser' cannot insert data into the 'salesDB' database.

In [ ]:
# Get connection string
connection = !atlas clusters connectionStrings describe MyNewCluster --projectId {project_id}

# Replace connection string with username and password
new_connection = connection[1].replace('mongodb+srv://', f'mongodb+srv://{salesReadUser}:{salesReadPass}@')

# Attempt to insert data
client = MongoClient(new_connection)
db = client['salesDB']
collection = db['mycollection']
try:
  data = {'name': 'John Doe', 'age': 30}
  result = collection.insert_one(data)
  print(f"Inserted document with ID: {result.inserted_id}")
except Exception as e:
  print(f"Error inserting data: {e}")

### 15. Test that 'myNewClusterAdmin' can insert data into the 'salesDB' database.

In [ ]:
# Get connection string
connection = !atlas clusters connectionStrings describe MyNewCluster --projectId {project_id}

# Replace 
new_connection = connection[1].replace('mongodb+srv://', f'mongodb+srv://{username}:{password}@')
client = MongoClient(new_connection)
db = client['salesDB']
collection = db['mycollection']
try:
  data = {'name': 'John Doe', 'age': 30}
  result = collection.insert_one(data)
  print(f"Inserted document with ID: {result.inserted_id}")
except Exception as e:
  print(f"Error inserting data: {e}")

## Encryption Challenge

### 16. Install the necessary packages.

In [ ]:
!pip install pymongo[encryption]
!pip install pymongo
!pip install requests

### 17. Set up the crypt_shared library.

In [ ]:
import os
from pymongo import MongoClient
from pymongo.encryption import Algorithm, ClientEncryption, QueryType
from pymongo.encryption_options import AutoEncryptionOpts
from bson.codec_options import CodecOptions
from bson import json_util
import json
import requests
import platform
import tempfile
def setup_crypt_shared():
    system = platform.system().lower()
    if system == "linux":
        url = "https://downloads.mongodb.com/linux/mongo_crypt_shared_v1-linux-x86_64-enterprise-ubuntu2004-6.0.6.tgz"
        filename = "mongo_crypt_shared_v1-linux-x86_64-enterprise-ubuntu2004-6.0.6.tgz"
    elif system == "darwin":
        url = "https://downloads.mongodb.com/osx/mongo_crypt_shared_v1-macos-x86_64-enterprise-6.0.6.tgz"
        filename = "mongo_crypt_shared_v1-macos-x86_64-enterprise-6.0.6.tgz"
    elif system == "windows":
        url = "https://downloads.mongodb.com/windows/mongo_crypt_shared_v1-windows-x86_64-enterprise-6.0.6.zip"
        filename = "mongo_crypt_shared_v1-windows-x86_64-enterprise-6.0.6.zip"
    else:
        raise OSError("Unsupported operating system")
    response = requests.get(url)
    response.raise_for_status()
    with tempfile.NamedTemporaryFile(delete=False, suffix=".tgz" if system != "windows" else ".zip") as tmp_file:
        tmp_file.write(response.content)
        tmp_file_path = tmp_file.name
    extract_dir = tempfile.mkdtemp()
    if system != "windows":
        os.system(f"tar -xzf {tmp_file_path} -C {extract_dir}")
        lib_path = os.path.join(extract_dir, "lib", "mongo_crypt_v1.so")
    else:
        os.system(f"powershell Expand-Archive -Path {tmp_file_path} -DestinationPath {extract_dir}")
        lib_path = os.path.join(extract_dir, "bin", "mongo_crypt_v1.dll")
    return lib_path
crypt_shared_lib_path = setup_crypt_shared()
print(f"Crypt shared library path: {crypt_shared_lib_path}")

### 18. Connect to your Atlas cluster.

In [ ]:
connection = !atlas clusters connectionStrings describe  MyNewCluster --projectId {project_id}
new_connection = connection[1].replace('mongodb+srv://', f'mongodb+srv://{username}:{password}@')
print(new_connection)
from pymongo import MongoClient
client = MongoClient(new_connection)

### 19. Set up an encrypted client using a local KMS provider.

In [ ]:
# generate a random 96 byte key
local_master_key = os.urandom(96)

# configure encryption options
kms_providers = {"local": {"key": local_master_key}}
key_vault_namespace = "encryption.__keyVault"
key_vault_database_name = "encryption"
key_vault_collection_name = "__keyVault"
key_vault_namespace = f"{key_vault_database_name}.{key_vault_collection_name}"

auto_encryption_options = AutoEncryptionOpts(
    kms_providers,
    key_vault_namespace,
    crypt_shared_lib_path=crypt_shared_lib_path
)

# set up client and encryption
encrypted_client = MongoClient(
    new_connection, auto_encryption_opts=auto_encryption_options)


client_encryption = ClientEncryption(
    kms_providers=kms_providers,
    key_vault_namespace=key_vault_namespace,
    key_vault_client=encrypted_client,
    codec_options=CodecOptions()
)

### 20. Consider the following sample 'patient' document:

In [ ]:
patient_document = {
    "patientName": "Jon Doe",
    "patientId": 12345678,
    "patientRecord": {
        "ssn": "987-65-4320",
        "billing": {
            "type": "Visa",
            "number": "4111111111111111",
        },
    },
}

### 21. Create an encrypted collection based on the following requirements:
- 'patientRecord.ssn' and 'billing' must be encrypted
- patients will be queried by 'patientRecord.ssn'

In [ ]:
# Create encrypted fields map, remeber that type is "string" and not "String" and "object" and not "Object"
encrypted_fields_map = <CODE_BLOCK>

encrypted_database_name = 'medicalRecords'
encrypted_collection_name = 'patients'
kms_provider_name = 'local'
client_encryption.create_encrypted_collection(
    encrypted_client[encrypted_database_name],
    encrypted_collection_name,
    encrypted_fields_map,
    kms_provider_name,
    {},
)

### 22. Insert the sample 'patient' document.

In [ ]:
encrypted_collection = encrypted_client[encrypted_database_name][encrypted_collection_name]
result = <CODE_BLOCK>
print(f"Inserted document ID: {result.inserted_id}")

### 22. Find the patient by its ssn

In [ ]:
find_result = <CODE_BLOCK>
print(find_result)

### 23. Make the same query the collection without encryption/decryption.

In [ ]:
reg_mongoclient = MongoClient(new_connection)
reg_collection = reg_mongoclient[encrypted_database_name][encrypted_collection_name] 
find_result = reg_colloection.find_one({"patientRecord.ssn": "987-65-4320"})
print(find_result)

### 24. Read all documents in the collection without encryption/decryption.

In [ ]:
all_docs = reg_mongoclient[encrypted_database_name][encrypted_collection_name].find()
print("\nAll documents in the collection:")
for doc in all_docs:
    print(doc)